In [1]:
import argparse
from collections import Counter
import code
import os
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [3]:
import numpy as np
import pandas as pd
import pkuseg
from elmoformanylangs import Embedder

In [4]:
e=Embedder('./zhs.model')
seg=pkuseg.pkuseg()


2020-01-20 16:39:47,146 INFO: char embedding size: 6169
2020-01-20 16:39:47,638 INFO: word embedding size: 71222
2020-01-20 16:39:51,295 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(71222, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(6169, 50, padding_idx=6166)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_feat

In [5]:
sents = ["今天天气真好啊", "潮水退了就知道谁没穿裤子"]
sents = [seg.cut(sent) for sent in sents]
print(sents)

[['今天', '天气', '真', '好', '啊'], ['潮水', '退', '了', '就', '知道', '谁', '没', '穿', '裤子']]


In [6]:
embeddings = e.sents2elmo(sents)

2020-01-14 02:05:12,001 INFO: 1 batches, avg len: 9.0


In [20]:
len(embeddings)  

2

In [21]:
embeddings[1].shape  # 句子2里有9个词,每个词是1024维的向量

(9, 1024)

In [24]:
candidate=[np.mean(embedding,0) for embedding in embeddings]
print(len(candidate[0]))

1024


In [7]:
def main():
    # for bash command
    #parser=argparse.ArgumentParser()
    #parser.add_argument("--train_file",default=None,type=str,required=True,
    #                   help="my train file")
    #parser.add_argument("--batch_size",default=64,type=int,required=False,
    #                   help="my batch size")
    #args=parser.parse_args()
    
    if not os.path.exists("embeddings.pkl"):
        # train_df = pd.read_csv(args.train_file)  for bash command
        train_df=pd.read_csv("/Users/valleria_ruka/Desktop/FAQ/lawzhidao_filter.csv")
        candidates=train_df[train_df["is_best"]==1][["title","reply"]]
        candidate_title=candidates["title"].tolist()
        candidate_reply=candidates["reply"].tolist()
        
        titles=[seg.cut(title) for title in candidates["title"]]
        embeddings=e.sents2elmo(titles)
        
        #可以不求平均,直接embedding送入
        # 这里是做了平均, 
        # size:  n个句子*每个句子里的单词*1024, 平均后 n个句子*1024
        candidate_embeddings=[np.mean(embedding,0) for embedding in embeddings]
        
        with open("embeddings.pkl","wb") as fout:
            # pickle.dump([candidate_title,candidate_reply,embeddings],fout)
            pickle.dump([candidate_title,candidate_reply,candidate_embeddings],fout)
    
    else:
         with open("embeddings.pkl","rb") as fin:
                #不求平均
                # candidate_title,candidate_reply,embeddings=pickle.load(fin)
                candidate_title,candidate_reply,candidate_embeddings=pickle.load(fin)
            
    
    while True:
        title=input("你的问题是? \n")
        if len(title.strip())==0:
            continue
        
        title=[seg.cut(title.strip())]
        title_embedding=[np.mean(e.sents2elmo(title)[0],0)]
        
        #不求平均
        # scores=cosine_similarity(title_embedding,embeddings)[0]
        scores=cosine_similarity(title_embedding,candidate_embeddings)[0]
        top5_indices=scores.argsort()[-5:][::-1]
        for index in top5_indices:
            print(print("可能的答案，参考问题：" + candidate_title[index] + 
                        "\t答案：" + candidate_reply[index] + "\t得分：" + str(scores[index])))
        

In [8]:
main()

你的问题是? 
怎样找律师


2020-01-14 17:24:54,975 INFO: 1 batches, avg len: 5.0


可能的答案，参考问题：做伤残鉴定一定要请律师吗	答案：不是伤残鉴定可以自行委托司法鉴定机构或者通过法院委托司法鉴定机构；律师只是受当事人委托代理办理伤残鉴定手续。参考伤残鉴定办理手续：一、需要携带证件：本人身份证、x光片、病历资料，工伤保险基金，治疗的病历资料；委托办理的需要委托书。二、具体手续：向法院或公安交通管理机关申请之后，获得批准后，持鉴定委托书申请人携带医院的诊断证明及出院证明及相关检查报告资料等相关材料前往批准机关指定的医疗鉴定机构进行鉴定。法医应于评定材料齐备后20日内做出《伤残评定书》。	得分：0.61108565
None
可能的答案，参考问题：如何咨询律师	答案：咨询可分为：现场口头咨询、当事人提供案件材料后律师提供咨询意见及法律方案；法律咨询是指提供法律知识问题的解答。由于法律的复杂性，非专业人士在遇到法律问题时，往往需要求助于律师一类的法律专业人士。律师收咨询费按计时收费，根据案件的复杂性和工作量及各地区不一样，知名度高的收费会高点，这个你要和律师协商，一般都是协商制。通常会根据问题的难易程度及回答问题所耗费的时间、精力评估收取费用。	得分：0.6047452
None
可能的答案，参考问题：我想找律师咨询一些问题.怎样联系到律师	答案：咨询可分为：现场口头咨询、当事人提供案件材料后律师提供咨询意见及法律方案；法律咨询是指提供法律知识问题的解答。由于法律的复杂性，非专业人士在遇到法律问题时，往往需要求助于律师一类的法律专业人士。律师收咨询费按计时收费，根据案件的复杂性和工作量及各地区不一样，知名度高的收费会高点，这个你要和律师协商，一般都是协商制。通常会根据问题的难易程度及回答问题所耗费的时间、精力评估收取费用。	得分：0.5949593
None
可能的答案，参考问题：想找个律师，哪里的律师好，怎么收费的	答案：各地区有一定差异，大致如下：无财产争议案件：普通民事、经济、行政案件，不涉及财产的，根据案件性质、复杂程度、工作所需耗费时间等因素，在6###—1#####元之间协商收取；外地民事、经济、行政案件不涉及财产的，代理费不低于2####元；律师代理分为以下几种第一种：一般代理。依律师职业道德为当事人代理案件，维护当事人权利。一般为纠纷标的的3-5%。标的高的话也可以低于该比例。第二种：风险代理。按胜诉金额或以得到的金额的百分比支付代理费

2020-01-14 17:25:00,890 INFO: 1 batches, avg len: 6.0


可能的答案，参考问题：给私人老板做工受伤?怎么赔偿,去找那个单位?	答案：打工受伤，可以选择私了，协商解决。如果不能协商，可以选择报警，需要到当地公安局做法医鉴定，再做赔偿	得分：0.73908746
None
可能的答案，参考问题：想工伤怎么算	答案：工伤赔偿的项目(一)治(医)疗费。治疗工伤所需费用必须符合工伤*险诊疗项目目录、工伤*险药品目录、工伤*险住院服*标准。(二)住院伙食补助费。职工住院治疗工伤的，由所在单位按照本单位因公出差伙食补助标准的70%发给住院伙食补助费。(三)外地就医交通费、食宿费。经医疗机构出具证明，报经办机构同意，工伤职工到统筹地区以外就医的，所需交通、食宿费用由所在单位按照本单位职工因公出差标准报销。(四)康复治疗费。工伤职工到签订服*协议的医疗机构进*康复性治疗的费用，符合工伤*险诊疗项目目录、工伤*险药品目录、工伤*险住院服*标准的本条第三款规定的，从工伤*险基金支付。(五)辅助器具费。工伤职工因日常生活或者就业需要，经*动能力鉴定委员会确认，可以安装假肢、矫形器、假眼、假牙和配置轮椅等辅助器具，所需费用按照国家规定的标准从工伤*险基金支付。(六)停工留薪期工资。职工因工作遭受事故伤害或者患职业病需要暂停工作接受工伤医疗的，在停工留薪期内，原工资福利待遇不变，由所在单位按月支付。(七)生活护理费。生活不能自理的工伤职工在停工留薪期需要护理的，由所在单位负责。工伤职工已经评定伤残等级并经*动能力鉴定委员会确认需要生活护理的，从工伤*险基金按月支付生活护理费。生活护理费按照生活完全不能自理、生活大*分不能自理或者生活*分不能自理3个不同等级支付，其标准分别为统筹地区上年度职工月平均工资的50%、40%或者30%。(八)一次性伤残补助金。一次性伤残补助金赔偿标准是根据工伤职员伤残等级确定，不同等级，赔偿标准不同。具体如下：一级伤残为27个月的本人工资，二级伤残为25个月的本人工资，三级伤残为23个月的本人工资，四级伤残为21个月的本人工资，五级伤残为18个月的本人工资，六级伤残为16个月的本人工资，七级伤残为13个月的本人工资，八级伤残为11个月的本人工资，九级伤残为9个月的本人工资，十级伤残为7个月的本人工资。(九)伤残津贴。职工因工致残被鉴定为一级至四级伤残的，从工伤*险基金按伤残等级支付一次性伤残补助金，标准为：一级伤残为24个月

2020-01-14 17:25:26,228 INFO: 1 batches, avg len: 6.0


可能的答案，参考问题：给私人老板做工受伤?怎么赔偿,去找那个单位?	答案：打工受伤，可以选择私了，协商解决。如果不能协商，可以选择报警，需要到当地公安局做法医鉴定，再做赔偿	得分：0.79969656
None
可能的答案，参考问题：咨询律师怎么找？	答案：咨询可分为：现场口头咨询、当事人提供案件材料后律师提供咨询意见及法律方案；法律咨询是指提供法律知识问题的解答。由于法律的复杂性，非专业人士在遇到法律问题时，往往需要求助于律师一类的法律专业人士。律师收咨询费按计时收费，根据案件的复杂性和工作量及各地区不一样，知名度高的收费会高点，这个你要和律师协商，一般都是协商制。通常会根据问题的难易程度及回答问题所耗费的时间、精力评估收取费用。	得分：0.79583985
None
可能的答案，参考问题：北京金石律师事务所怎么样？收费贵吗？如何找律师？	答案：各地区有一定差异，大致如下：无财产争议案件：普通民事、经济、行政案件，不涉及财产的，根据案件性质、复杂程度、工作所需耗费时间等因素，在6###—1#####元之间协商收取；外地民事、经济、行政案件不涉及财产的，代理费不低于2####元；律师代理分为以下几种第一种：一般代理。依律师职业道德为当事人代理案件，维护当事人权利。一般为纠纷标的的3-5%。标的高的话也可以低于该比例。第二种：风险代理。按胜诉金额或以得到的金额的百分比支付代理费，或按减少的支付额的百分比支付代理费，比例当然高于一般代理。第三种：半风险代理。给一定的基本费用，其他的再按风险代理计算，当然就介于一般代理和风险代理的比例之间。	得分：0.78619725
None
可能的答案，参考问题：我想找律师咨询一些问题.怎样联系到律师	答案：咨询可分为：现场口头咨询、当事人提供案件材料后律师提供咨询意见及法律方案；法律咨询是指提供法律知识问题的解答。由于法律的复杂性，非专业人士在遇到法律问题时，往往需要求助于律师一类的法律专业人士。律师收咨询费按计时收费，根据案件的复杂性和工作量及各地区不一样，知名度高的收费会高点，这个你要和律师协商，一般都是协商制。通常会根据问题的难易程度及回答问题所耗费的时间、精力评估收取费用。	得分：0.77493465
None
可能的答案，参考问题：浠水人我想问下起诉离婚要怎么处理需要找律师吗	答案：委托律师是公民的权利，律师费用参考：第一

2020-01-14 17:26:03,022 INFO: 1 batches, avg len: 6.0


可能的答案，参考问题：给私人老板做工受伤?怎么赔偿,去找那个单位?	答案：打工受伤，可以选择私了，协商解决。如果不能协商，可以选择报警，需要到当地公安局做法医鉴定，再做赔偿	得分：0.810823
None
可能的答案，参考问题：做工伤鉴定应该怎么弄，需要什么材料吗	答案：1、《工伤保险条例》第十七条规定：职工发生事故伤害或者按照职业病防治法规定被诊断、鉴定为职业病，所在单位应当自事故伤害发生之日或者被诊断、鉴定为职业病之日起30日内，向统筹地区劳动保障行政部门提出工伤认定申请。遇有特殊情况，经报劳动保障行政部门同意，申请时限可以适当延长。用人单位未按前款规定提出工伤认定申请的，工伤职工或者其直系亲属、工会组织在事故伤害发生之日或者被诊断、鉴定为职业病之日起1年内，可以直接向用人单位所在地统筹地区劳动保障行政部门提出工伤认定申请。按照本条第一款规定应当由省级劳动保障行政部门进行工伤认定的事项，根据属地原则由用人单位所在地的设区的市级劳动保障行政部门办理。用人单位未在本条第一款规定的时限内提交工伤认定申请，在此期间发生符合本条例规定的工伤待遇等有关费用由该用人单位负担。2、《工伤保险条例》第十八条规定：提出工伤认定申请应当提交下列材料：（一）工伤认定申请表；（二）与用人单位存在劳动关系（包括事实劳动关系）的证明材料；（三）医疗诊断证明或者职业病诊断证明书（或者职业病诊断鉴定书）。工伤认定申请表应当包括事故发生的时间、地点、原因以及职工伤害程度等基本情况。工伤认定申请人提供材料不完整的，劳动保障行政部门应当一次性书面告知工伤认定申请人需要补正的全部材料。申请人按照书面告知要求补正材料后，劳动保障行政部门应当受理。3、《工伤保险条例》第二十一条规定：职工发生工伤，经治疗伤情相对稳定后存在残疾、影响劳动能力的，应当进行劳动能力鉴定。根据上述法律规定：您可以自己依法申请工伤认定，认定工伤后，再依法进行劳动能力鉴定，确定伤残等级，这样，就可以依法享受工伤医疗、停工留薪、工伤伤残待遇了。	得分：0.8042079
None
可能的答案，参考问题：360.借钱是真的吗?不是高利贷坑人的?	答案：民间借贷是一种民事法律行为合法的在法律上认定的高利。贷是指超过银行同期基准利率的4倍，也就是产生法律纠纷时支持同期基准利率四倍以内的利率水平，超过银行同期基准利率的4倍，即为高利。贷，不受法

KeyboardInterrupt: 

In [7]:
def mean_reciprocal_rank(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])


def evaluate_mmr():
    # load candidate embeddings
    with open("embeddings.pkl","rb") as fin:
        candidate_title,candidate_reply,candidate_embeddings=pickle.load(fin)
    
    
    # load test data
    df=pd.read_csv("/Users/valleria_ruka/Desktop/FAQ/lawzhidao_evaluate.csv")
    questions=df["question"].tolist()
    matched_questions=df["title"].tolist()
    matched_questions_index = []
    for q in matched_questions:
        flg = False
        for i, _q in enumerate(candidate_title):
            if q == _q:
                matched_questions_index.append([i])
                flg = True
                break
        if flg == False:
            matched_questions_index.append([-1])
    
    matched_questions_index = np.asarray(matched_questions_index)
    
    
    questions = [seg.cut(q.strip()) for q in questions]
    question_embedding = [np.mean(emb, 0) for emb in e.sents2elmo(questions)]


    scores = cosine_similarity(question_embedding, candidate_embeddings)
    sorted_indices = scores.argsort()[:, ::-1]#[-5:][::-1]

    mmr = mean_reciprocal_rank(sorted_indices==matched_questions_index)
    print("mean reciprocal rank: {}".format(mmr))

In [8]:
evaluate_mmr()

2020-01-20 16:41:03,312 INFO: 1 batches, avg len: 6.0


mean reciprocal rank: 0.1978919797772126
